## Data integration script

The aim of this task is to integrate in the same file the following data:
- GPS;
- Weather;
- Waze;
- GTFS

Example of final file attributes (to use in ML algorithms):


The task has the following steps:
1. Clean the data (removing missing/wrong data);
2. Label shape file with route type (low/high frequency)
3. Update and run BULMA (matching of GPS and GTFS);
4. Update and run BUSTE (interpolate stops timestamp)
5. Label each GPS with headway value and BB (headway, BB, id_bus_bb)
6. Label with precipitation (precipitation)

In [13]:
import utm
import os
import pandas as pd
from datetime import datetime
from numpy import median
import math

### Data example

#### GPS
- bus_code
- timestamp
- route
- latitude
- longitude

#### Weather

#### Waze

#### GTFS

Pre-processing GPS data of Recife to separate files per day and to convert coordinates.

In [ ]:
def convertCoordinates(x, y):
    if (x == '-' or x == '0' or y == '-' or y == '0'):
        return ['-', '-']
    
    return utm.to_latlon(long(x), long(y), 25, 'M')
    
def separateGPSFilePerMonth(file_path):
    DELIMITER = ','
    october_file = open(file_path + 'GPS_data_october.csv', 'w')
    november_file = open(file_path + 'GPS_data_november.csv', 'w')
    december_file = open(file_path + 'GPS_data_december.csv', 'w')
    columns_name = "Unidad" + DELIMITER +  "Instante" + DELIMITER + "Estado" + DELIMITER + "Comunica" + DELIMITER + "CoordX" + DELIMITER + "CoordY" + DELIMITER + "Linea" + DELIMITER + "Ruta" + DELIMITER + "Posicion" + DELIMITER + "Viaje" + DELIMITER + "Velocidad"
    october_file.write(columns_name + '\n')
    november_file.write(columns_name + '\n')
    december_file.write(columns_name + '\n')
    
    october = '2018-10'
    november = '2018-11'
    december = '2018-12'
    
    file_name = file_path + "GPS_data.csv"
    with open(file_name, 'r') as gps_data:
        next(gps_data)
        for line in gps_data:
            line_splitted = line.split(',')
            date = line_splitted[1]
            lat = line_splitted[4]
            lon = line_splitted[5]
            new_coordinates = convertCoordinates(lat, lon)
            
            new_line = line_splitted[0] + DELIMITER + line_splitted[1] + DELIMITER + line_splitted[2] + DELIMITER + line_splitted[3] + DELIMITER + str(new_coordinates[0]) + DELIMITER + str(new_coordinates[1]) + DELIMITER + line_splitted[6] + DELIMITER + line_splitted[7] + DELIMITER + line_splitted[8] + DELIMITER + line_splitted[9] + DELIMITER + line_splitted[10]
            
            if (october in date):
                october_file.write(new_line)
            elif (november in date):
                november_file.write(new_line)
            elif (december in date):
                december_file.write(new_line)

def separateGPSFilePerDay(file_path):
    DELIMITER = ','
    columns_name = "Unidad" + DELIMITER +  "Instante" + DELIMITER + "Estado" + DELIMITER + "Comunica" + DELIMITER + "CoordX" + DELIMITER + "CoordY" + DELIMITER + "Linea" + DELIMITER + "Ruta" + DELIMITER + "Posicion" + DELIMITER + "Viaje" + DELIMITER + "Velocidad"
    file_name = file_path + "GPS_data_october.csv"
    date_lines_dict = {}
    with open(file_name, 'r') as gps_data:
        next(gps_data)
        for line in gps_data:
            line_splitted = line.split(',')
            date_time = line_splitted[1]
            date = date_time.split(' ')[0]
            
            if (date not in date_lines_dict):
                date_lines_dict[date] = []
            
            date_lines_dict[date].append(line)
            
    
    for key in date_lines_dict:
        new_file = open(file_path + 'GPS_data_' + key + '.csv', 'w')
        new_file.write(columns_name + '\n')
        
        for data_per_day in date_lines_dict[key]:
            new_file.write(data_per_day)
            
        new_file.close()
            
file_path = os.getcwd() + "/../data/input/Recife/GPS/"

separateGPSFilePerMonth(file_path)

separateGPSFilePerDay(file_path)

### 1. Clean the data
Removing missing/wrong data

#### 1.1 Waze

In [ ]:
# Waze
# Removing lines from different city

def removeDifferentCity(dir_name, city_label):
    
    for file_name in os.listdir(dir_name):
        
        if file_name.endswith(".csv"): # to get just files
            file_path = dir_name + file_name

            new_file = open(dir_name + 'clean/' + file_name, 'w')

            with open(file_path, 'r') as waze_data:
                new_file.write(next(waze_data)) # write the header
                
                for line in waze_data:
                    line_splitted = line.split(',')
                    city = line_splitted[1]

                    if (city_label in city):
                        new_file.write(line) # to add just lines of the city

                new_file.close()
            
# Alert: create the folder 'clean', set the path and the city name            
dir_path = os.getcwd() + "/../data/input/Recife/Waze/"
city = 'Recife'
removeDifferentCity(dir_path, city)

#### 1.2 GPS

Removing Recife data without *linea* (route) or *CoordX* or *CoordY*

In [ ]:
# GPS - Recife
# Removing lines without linea or CoordX or CoordY

def cleanRecifeGPS(dir_path):
    for file_name in os.listdir(dir_path):
        
        if file_name.endswith(".csv"): # to get just files
            file_path = dir_path + file_name
            
            new_file = open(dir_path + 'clean/' + file_name, 'w')

            with open(file_path, 'r') as gps_data:
                new_file.write(next(gps_data)) # write the header
                
                for line in gps_data:
                    line_splitted = line.split(',')
                    route = line_splitted[6]
                    lat = line_splitted[4]
                    lon = line_splitted[5]

                    if (route != '' and lat != '-' and lat != '' and lon != '-' and lon != ''):
                        new_file.write(line) # to add just lines not empty

                new_file.close()
                
# Alert: create the folder 'csv' 
dir_path = os.getcwd() + "/../data/input/Recife/GPS/"
cleanRecifeGPS(dir_path)

In [ ]:
# GPS - Curitiba
# Converting json file to csv file

# Alert: first, replace }{ by },{ 
# sed -i ':a;N;$!ba;s/}/},/g' *.json

def convertJSON2CSV(dir_path):
    DELIMITER = ','
    columns_name = "bus_code" + DELIMITER + "lat" + DELIMITER + "lon" + DELIMITER + "timestamp" + DELIMITER + "route"
    
    for file_name in os.listdir(dir_path):
        
        if file_name.endswith(".json"): # to get just files
            file_path = dir_path + file_name
            
            new_file = open(dir_path + 'csv/' + file_name.split('.')[0] + '.csv', 'w')

            new_file.write(columns_name + "\n")
            
            with open(file_path, 'r') as gps_data:
                next(gps_data) # skip empty line 
                for line in gps_data:
                    line_splitted = line.split(',')
                    bus_code = line_splitted[0].split('":')[1]
                    lat = line_splitted[1].split('":')[1]
                    lon = line_splitted[2].split('":')[1]
                    timestamp = line_splitted[3].split('":')[1]
                    route = line_splitted[4].split('":')[1][:-1]
                    
                    new_line = bus_code + DELIMITER + lat + DELIMITER + lon + DELIMITER + timestamp + DELIMITER + route
                    
                    new_file.write(new_line + "\n")
            
dir_path = os.getcwd() + "/../data/input/Curitiba/GPS/"
convertJSON2CSV(dir_path)

### 2. Label shape file with route type

Label shape file with route type: high frequency or low frequency based on *headway median of the city (h_median)*.

- **High frequency:** headway mean of the route <= h_median
- **Low frequency:** headway mean of the route > h_median

<img src="trips_file_example.png">

The *stop_times* file has the information:
- Buses: all the schedules of the bus are grouped, e.g. the schedules of the first bus are the firsts lines (from 5:00 to 23:00)
- Headway: the diference of the *arrival_time* of the bus and the follow.

**Challenge: how to separate different buses to compare headways?**

In [2]:
# Alert: Set the path with the city name            
dir_path = os.getcwd() + "/../data/input/Curitiba/GTFS/"
trips_file = 'trips.txt'
stop_times_file = 'stop_times.txt'

# Read trips.txt to get the route of the trips
def getRouteTripsMap():
    route_trips_map = {}
    
    with open(dir_path + trips_file, 'r') as trips_data:
        next(trips_data) # skip header
        
        for line in trips_data:
            splitted_line = line.split(',')
            route = splitted_line[0]
            trip_id = splitted_line[2]
            
            if (route not in route_trips_map):
                route_trips_map[route] = []
            
            route_trips_map[route].append(trip_id)
    
#     print(route_trips_map)
    
    return route_trips_map
    

# Read stop_times.txt to get the stop_times of each trip
def getTripsStopsTimesMap(route_trips_map):
    route_stops_times = {}
    
    with open(dir_path + stop_times_file, 'r') as stops_times_data:
        next(stops_times_data) # skip header
        
        for line in stops_times_data:
            splitted_line = line.split(',')
            trip_id = splitted_line[0]
            stop_id = splitted_line[3]
            arrival_time = splitted_line[1]
            
            # get the route of the trip 
            for key, trips in route_trips_map.items():
                if trip_id in trips:
                    route = key

            if (route not in route_stops_times):
                route_stops_times[route] = {}
            
            if (stop_id not in route_stops_times[route]):
                route_stops_times[route][stop_id] = []
            
            route_stops_times[route][stop_id].append(arrival_time)
                    
#     print(route_stops_times['34'])
    return route_stops_times


# Calculate the headway(difference between two timestamps) in seconds
def getHeadway(time1, time2):
    t1 = datetime.strptime(time1, "%H:%M:%S")
    t2 = datetime.strptime(time2, "%H:%M:%S")
    difference = t2 - t1
    
    return difference.seconds


In [3]:
route_trips_map = getRouteTripsMap()
route_stops_times = getTripsStopsTimesMap(route_trips_map)

In [17]:
# Calculate headway median of each route;
# headway median of the city.
def calculateRoutesHeadways(route_stops_times):
    route_headway_mean_map = {}
    sum_headways_city = []
    
    for route, stopstimes in route_stops_times.items():
        
        sum_headways_route = []
        
        for stops, times in stopstimes.items():
            last_time = 0
            
            # Order times before to iterate
            ordered_times = sorted(times, key=lambda d: map(int, d.split(':')))
            for time in ordered_times:
                
                # skip the first timestamp or wrong data
                if (last_time != 0 and not time.startswith('24:') and not last_time.startswith('24:')): 
                    # mean doesn't work because some routes run in specific times
                    headway = getHeadway(last_time, time)
                    
                    sum_headways_route.append(headway)
                    
                last_time = time
                
        # calculate the median headway of the route
        if (len(sum_headways_route) > 0):
            headway_route = median(sum_headways_route)
            route_headway_mean_map[route] = headway_route
            
            sum_headways_city.append(headway_route)
        
    median_headway_city = median(sum_headways_city)
    return (median_headway_city, route_headway_mean_map)
    
calculateRoutesHeadways(route_stops_times)

(1413.0,
 {'1': 900.0,
  '100': 1200.0,
  '102': 1320.0,
  '105': 2242.0,
  '106': 1560.0,
  '110': 1920.0,
  '112': 954.0,
  '113': 1845.0,
  '114': 960.0,
  '115': 1320.0,
  '116': 360.0,
  '12': 2040.0,
  '122': 7200.0,
  '123': 1200.0,
  '125': 1200.0,
  '126': 1232.0,
  '127': 1267.0,
  '128': 1374.0,
  '132': 1920.0,
  '137': 1200.0,
  '139': 1080.0,
  '141': 1145.0,
  '142': 1466.0,
  '143': 1080.0,
  '144': 1377.0,
  '145': 2700.0,
  '146': 1366.0,
  '147': 2040.0,
  '148': 1056.5,
  '150': 840.0,
  '151': 900.0,
  '153': 1128.0,
  '156': 900.0,
  '16': 1612.0,
  '162': 900.0,
  '164': 1440.0,
  '17': 639.0,
  '170': 1440.0,
  '171': 1200.0,
  '173': 660.0,
  '174': 2400.0,
  '175': 1560.0,
  '176': 720.0,
  '177': 780.0,
  '178': 1565.0,
  '179': 431.0,
  '180': 840.0,
  '181': 1200.0,
  '182': 1200.0,
  '183': 2713.0,
  '185': 945.0,
  '186': 1200.0,
  '187': 1260.0,
  '188': 1200.0,
  '189': 1500.0,
  '190': 1200.0,
  '191': 1276.0,
  '192': 1320.0,
  '193': 1200.0,
  '195':

In [ ]:
#TODO read routes.txt and update with high/low frequency 

Each stop time belongs to one trip, i.e. the total os trips is the total os time stamps there.

**Observation:**

- In general, the scheduled headway of Curitiba is 1413 seconds (23 minutes).
- In general, the scheduled headway of Campina Grande is 1413 seconds (23 minutes).
- In general, the scheduled headway of Recife is 1413 seconds (23 minutes).